In [1]:
import json
from bs4 import BeautifulSoup as bs
import requests
import pprint
import time
import random
import re
import numpy as np
import pandas as pd
from pymongo import MongoClient

import warnings
warnings.filterwarnings('ignore')

### Задание с дз-03:  
#### 1) Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта job.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:   
* Наименование вакансии,  
* Предлагаемую зарплату,  
* Ссылку на саму вакансию  
  
#### 2) Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)  

### Задание дз-04:  
#### 1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.  
#### 2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы  
#### 3*) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

# Конфиг:

In [2]:
# Укажите здесь вакансию которую вы ищете
JOB_NAME = "программист"

# Укажите сколько страниц с вакансиями достаточно пропарсить 
# (если страниц с результатами будет меньше чем заданное число, то будут пропарсены все страницы):
pages_enough = 35

# Искать только вакансии с указанными зарплатами (если false, ищутся все вакансии):
only_with_salary=True

In [3]:
headers = requests.utils.default_headers()
headers['User-Agent'] = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36"

In [4]:
def make_link_for_jobname(jobname_string, only_with_salary=False):
    jobname_string = jobname_string.replace("+", "%2B")
    jobname_string = jobname_string.replace("#", "%23")
    jobname_split = jobname_string.split(" ")
    only_with_salary = "true" if only_with_salary else "false"
    link = "https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&only_with_salary=%s&clusters=true&enable_snippets=true&text=%s&page=0" % (only_with_salary, "+".join(jobname_split))
    return link

In [5]:
def find_next_page_link(page_content):
    if page_content.find('a', {"class": "bloko-button HH-Pager-Controls-Next HH-Pager-Control"}, href=True) is None:
        return False
    else:
        return "https://hh.ru" + page_content.find('a', {"class": "bloko-button HH-Pager-Controls-Next HH-Pager-Control"}, href=True)['href']

In [6]:
def find_all_vacancies_items(parsed_html):
    return parsed_html.find_all('div', {'class': 'vacancy-serp-item'})

In [7]:
def parse_vacancy_item(vacancy_item, debug=False):
    
    vacancy_item_second_child = vacancy_item.findChild().find_next_sibling()
    vacancy_item_third_child = vacancy_item.findChild().find_next_sibling().find_next_sibling()

    tmp_jobname = vacancy_item_second_child.findChild().getText()
    if debug:
        print("=============================================================================")
        print("Название:\t", tmp_jobname, flush=True)
        
    tmp_vacancy_href = vacancy_item_second_child.findChild(href=True)['href']
    if debug:
        print("Ссылка:\t", tmp_vacancy_href, flush=True)
        
    tmp_value = vacancy_item_second_child.findChild().find_next_sibling().getText()
    tmp_value2 = {"min": 0, "max": 0, "currency": "руб."}
    if tmp_value != "":
        tmp_value = re.sub(u'\xa0', "", tmp_value)
        if tmp_value.startswith("от"):
            tmp_value2["min"] = int(tmp_value.split(" ")[1])
            tmp_value2["max"] = 9999999999
            tmp_value2["currency"] = tmp_value.split(" ")[2]
        elif tmp_value.startswith("до"):
            tmp_value2["min"] = 0
            tmp_value2["max"] = int(tmp_value.split(" ")[1])
            tmp_value2["currency"] = tmp_value.split(" ")[2]
        else:
            tmp_value_split = tmp_value.split(" ")
            tmp_value2["currency"] = tmp_value_split[1]
            tmp_value_split = tmp_value_split[0].split("-")
            tmp_value2["min"] = int(tmp_value_split[0])
            tmp_value2["max"] = int(tmp_value_split[1])
    
    if debug:
        print("Зарплата:\t", tmp_value, flush=True)
        
    tmp_obyazan = vacancy_item.find("div", {"data-qa": "vacancy-serp__vacancy_snippet_responsibility"}).getText()
    if debug:
        print("Обязанности:\t", tmp_obyazan, flush=True)
        
    tmp_requir = vacancy_item.find("div", {"data-qa": "vacancy-serp__vacancy_snippet_requirement"}).getText()
    if debug:
        print("Требования:\t", tmp_requir, flush=True)
        print("=============================================================================")

    return {"job": tmp_jobname, "link": tmp_vacancy_href, "value": tmp_value2, "response": tmp_obyazan, "req": tmp_requir}

In [8]:
link = make_link_for_jobname(JOB_NAME, only_with_salary=only_with_salary)
all_job_offers = []
pages_counter = 0

while link:
#     print(link)
    get_ = requests.get(link, headers=headers)
    get_.encoding = 'utf-8'
    html = get_.text
    parsed_html = bs(html,'lxml')
    link = find_next_page_link(parsed_html)
    all_vacancies = find_all_vacancies_items(parsed_html)
    for vacancy_item in all_vacancies:
        all_job_offers.append(parse_vacancy_item(vacancy_item))
    pages_counter += 1
    print("\r#%d/%d страниц пропарсено" % (pages_counter, pages_enough), end = "", flush=True)
    if pages_counter == pages_enough:
        link = False
print()

#35/35 страниц пропарсено


In [9]:
with open("vacancies.txt", "wt", encoding="utf-8") as output_file:
    if len(all_job_offers) > 0:
        for rec in all_job_offers:
            #{"job": tmp_jobname, "link": tmp_vacancy_href, "value": tmp_value, "response": tmp_obyazan, "req": tmp_requir}
            output_file.write("Вакансия:\n\t" + rec["job"] + "\n")
            output_file.write("Зарплата:\n\tВалюта: " + rec["value"]["currency"] + "\n\t" + "Мин.: " + str(rec["value"]["min"]) + "\n\tМакс.: " + str(rec["value"]["max"]) + "\n")
            output_file.write("Ссылка:\n\t" + rec["link"] + "\n")
            output_file.write("Обязанности:\n\t" + rec["response"] + "\n")
            output_file.write("Требования:\n\t" + rec["req"] + "\n")
            output_file.write("=====================================================================================\n\n")
    else:
        output_file.write("Вакансий по запросу \"%s\" не найдено.\n" % JOB_NAME)
            

In [10]:
# with open("vacancies.txt", "rt", encoding="utf-8") as file:
#     print(file.read())

In [11]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['headhunter']
hhdb = db.headhunter

In [12]:
# Drop database:
# client.drop_database("headhunter")

### Я подумал, что сама база MongoDB лучше и быстрее у себя внутри определит, что я пытаюсь ей подсунуть дубликат, поэтому мне показалось, что быстрее обработать возвращаемые исключения, и пропускать их если это "DuplicateKeyError", так что добавляются в итоге только новые вакансии.

In [13]:
for item in all_job_offers:
#     print(item["value"])
    try:
        hhdb.insert(item)
    except Exception as e:
        e_class_name = e.__class__.__name__
        if e_class_name != "DuplicateKeyError":
            print("WARNING: ERROR!\n\t", e, "\n\tFor item:\n\t", item)

In [14]:
def show_what_in_database(print_limit=0):
    cursor = hhdb.find()
    i = 0
    for document in cursor:
        pprint.pprint(document)
        print()
        i += 1
        if print_limit != 0:
            if i == print_limit:
                return

In [15]:
show_what_in_database(print_limit=5)

{'_id': ObjectId('5d25c5b0dbf510c2be3bd3cf'),
 'job': 'Старший разработчик С++/Qt',
 'link': 'https://tomsk.hh.ru/vacancy/32375667?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'req': 'Понимание принципов ООП. Знание структур и алгоритмов обработки '
        'данных. Знание С++. Знание паттернов проектирования. Хорошее знание '
        'библиотеки Qt5 (!). ',
 'response': 'Участие в разработке и испытаниях ПО. Проект по разработке '
             'кросс-платформенного коммуникационного клиента, включающего в '
             'себя функционал почтового клиента, работы...',
 'value': {'currency': 'руб.', 'max': 180000, 'min': 140000}}

{'_id': ObjectId('5d25c5b0dbf510c2be3bd3d0'),
 'job': 'Программист 1C',
 'link': 'https://tomsk.hh.ru/vacancy/27844837?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'req': 'Опыт профессиональной разработки на платформе 1С:Предприятие 8. Опыт '
        'разработки в клиент-серверном режиме, умение раб

### В вакансиях встречаются разные валюты:

In [16]:
print(*hhdb.distinct("value.currency"), sep="\n")

руб.
EUR
USD


In [17]:
def nice_print_vacancy(rec):
    print("Вакансия:\n\t" + rec["job"])
    print("Зарплата:\n\tВалюта: " + rec["value"]["currency"] + "\n\t" + "Мин.: " + str(rec["value"]["min"]) + "\n\tМакс.: " + str(rec["value"]["max"]))
    print("Ссылка:\n\t" + rec["link"])
    print("Обязанности:\n\t" + rec["response"])
    print("Требования:\n\t" + rec["req"])
    print("=============================================================================================================\n")

In [18]:
def find_vacancies_with_big_zp(zp_value=0, currency="руб."):
    for res in hhdb.find({"value.min": {"$gt": zp_value}, "value.currency": {"$eq": currency}}):
        nice_print_vacancy(res)


In [21]:
# Найти зарплату больше чем "ZP_VALUE" в валюте "CURRENCY":
ZP_VALUE = 500

# CURRENCY = "EUR"
# CURRENCY = "руб."
CURRENCY = "USD"

In [22]:
find_vacancies_with_big_zp(zp_value=ZP_VALUE, currency=CURRENCY)

Вакансия:
	Ведущий разработчик/разработчик C++/Qt (GUI Team, VMS)
Зарплата:
	Валюта: USD
	Мин.: 4000
	Макс.: 9999999999
Ссылка:
	https://tomsk.hh.ru/vacancy/28163901?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
Обязанности:
	Программировать GUI на Qt Widgets, QML, а иногда и на OpenGL. Постоянно улучшать существующий код. Нет предела совершенству. 
Требования:
	Знание сетевых протоколов и основ работы с базами данных. Иногда GUI-разработчикам приходится писать и серверный код. Опыт разработки мобильных...

Вакансия:
	Senior Scala/Akka Developer
Зарплата:
	Валюта: USD
	Мин.: 3000
	Макс.: 4500
Ссылка:
	https://tomsk.hh.ru/vacancy/32229113?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
Обязанности:
	
Требования:
	Отличное знание Java, Scala, понимание архитектуры и особенностей JVM. - Опыт разработки распределенных систем с использованием Akka. - Глубокие знания архитектуры компьютерных...

Вакансия:
	Senior Full-Stack Developer / Java